In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Load the data
df=pd.read_csv("../data/transfers_test.csv")

In [5]:
df

,type,txhash,timestamp,amount,denom,dpDenom,fromAddress,toAddress,fromChain,fromChainId,toChain,toChainId
0,Send,413D0BB67CBC4A66B2C32ECDD1C5974BCB6BBF012D6C7F...,1667135085000,7.216000e+00,uatom,ATOM,cosmos1gwyv83zcnckdhuz3n78rvyzj59u8x6l8dk9cfy,cosmos1v0zv6s67mt7hqv6ky7au3pkqw3xcpp78pnp4uj,cosmos,cosmoshub-4,cosmos,cosmoshub-4
1,Send,931A14D99B42D18EA2BD5C61105489880BF1F7939A758B...,1667135085000,6.556260e+00,uatom,ATOM,cosmos153lf4zntqt33a4v0sm5cytrxyqn78q7kz8j8x5,cosmos1q8mwcpw4x0kpskajuglgkhevpunujy3c2ekyh6,cosmos,cosmoshub-4,cosmos,cosmoshub-4
2,Send,ADA7DE328D1C68DA26F9721FCB407A41059C79F0C83C06...,1667135085000,3.000000e+00,uatom,ATOM,cosmos1k77ggtxdd7fnahadu9nycpjmlnt87aq4cttcgx,cosmos1psr5x3kgra5fvm4gc4l6ufykn0nl3esdjeex8n,cosmos,cosmoshub-4,cosmos,cosmoshub-4
3,Send,08C37BBC858078112C461D81842F0B64C765E0FEC03946...,1667135086000,6.498000e+02,uscrt,SCRT,secret1an5pyzzpu5ez7ep9m43yzmalymwls39qtk8rjd,secret1y9chu4ee326g9ay02nyv2sh7hhquz9sx52k785,secret,secret-4,secret,secret-4
4,Send,C8310F1D40F50A019BD76F5862E7B8D9B36DF4E33F3672...,1667135086000,5.000000e+02,inj,INJ,inj17lw6c2xxflhljt3sty3sch9kpvfhcz98apdj8d,inj1u2rajhqtptzvu23leheta9yg99k3hazf4waf43,injective,injective-1,injective,injective-1
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,IBCReceive,479D64046F9CD863EB12028FBA8FC747AF0D5D2151656D...,1667210671000,1.245578e+15,aevmos,Unknown,evmos1h36d5tsscpkyk9hp2f5zaag6jgaafnj6svref2,osmo183vg0sp0z2wg02pnpfqut6fmmgu2pdy7dnq2cg,evmos,evmos_9001-2,osmosis,osmosis-1
99996,Send,50ADD266CCB2B0BA07D4298F457C242DD78B0DBB7885D4...,1667210671000,4.769990e+02,uosmo,OSMO,osmo1krmqr8zhvteevq4hnl7q0pxtff6xpnqz4ckxcu,osmo1hsym9q72n4tvm3e5tz8l3ua5jg30mwrsx4wj2e,osmosis,osmosis-1,osmosis,osmosis-1
99997,Send,8FD3D90526321023DF187E3D151E3B08C6E5F59371DAA5...,1667210671000,6.000000e+00,aevmos,EVMOS,evmos10rhty7em3tdckwkwn75ed8m9ftn4llychmf8lm,evmos1dac4dgmn26r3yrd3pnerzmmdfkau4092t8gk56,evmos,evmos_9001-2,evmos,evmos_9001-2
99998,Send,C4CD1140DC699434DC729EC8BE3190E97B972854DBE237...,1667210671000,5.700000e-01,uosmo,OSMO,osmo1z46cheg6cg9s46nv9vfkr2cedcmu204f5muyfs,osmo19ulkyzen85zqvjmyjcv3454a9r5t8yfn9zk4sz,osmosis,osmosis-1,osmosis,osmosis-1


In [6]:
# IBCReceive 제거
df = df[df['type'] != 'IBCReceive']

In [7]:
# 외부체인 거래인가?
df['is_external'] = df['fromChain'] != df['toChain']

/tmp/ipykernel_115274/4245052492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_external'] = df['fromChain'] != df['toChain']


In [8]:
node_df = df

In [9]:
external_df = node_df[node_df['is_external']]

In [10]:
# 1. 보낸 트랜젝션 집계
sent_stats = (
    node_df
    .groupby('fromAddress')
    .agg(sent_tx_count = ('amount','count'),
         sent_tx_amount = ('amount','sum'))
    .rename_axis('address')
)

# 2. 받은 트랜젝션 집계
recv_stats = (
    node_df
    .groupby('toAddress')
    .agg(recv_tx_count = ('amount','count'),
         recv_tx_amount = ('amount','sum'))
    .rename_axis('address')
)

# 3. external sent 거래 집계
external_sent_stats = (
    external_df
    .groupby('fromAddress')
    .agg(external_sent_tx_count = ('amount','count'),
         external_sent_tx_amount = ('amount','sum'))
    .rename_axis('address')
)
# 4. external recv 거래 집계
external_recv_stats = (
    external_df
    .groupby('toAddress')
    .agg(external_recv_tx_count = ('amount','count'),
         external_recv_tx_amount = ('amount','sum'))
    .rename_axis('address')
)

# 1~4 종합
total_stats = (
    sent_stats
    .join(recv_stats, how='outer')
    .join(external_sent_stats, how='outer')
    .join(external_recv_stats, how='outer')
)

In [11]:
total_stats.describe()

,sent_tx_count,sent_tx_amount,recv_tx_count,recv_tx_amount,external_sent_tx_count,external_sent_tx_amount,external_recv_tx_count,external_recv_tx_amount
count,28196.000000,2.819600e+04,20197.000000,2.019700e+04,10784.000000,1.078400e+04,10011.000000,1.001100e+04
mean,2.703540,3.609861e+07,3.774273,5.039543e+07,1.814076,1.053462e+06,1.954150,1.134805e+06
std,37.977071,5.955773e+09,52.735100,7.037014e+09,11.772387,1.017334e+08,5.570885,1.056379e+08
min,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00
25%,1.000000,1.280000e+00,1.000000,1.793000e+00,1.000000,1.500000e+00,1.000000,1.796143e+00
50%,1.000000,4.200000e+00,1.000000,1.107777e+01,1.000000,7.338183e+00,1.000000,9.272164e+00
75%,1.000000,2.230000e+01,2.000000,9.032000e+01,1.000000,5.358465e+01,2.000000,7.100000e+01
max,1812.000000,1.000000e+12,4642.000000,1.000000e+12,986.000000,1.055000e+10,309.000000,1.055500e+10


In [12]:
# 5. 시간대별 entrophy 계산
temp_df = node_df
# 5-1. timestamp → hour
temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp'], unit = 'ms')
temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
temp_df['hour'] = temp_df['timestamp'].dt.hour

# 5-2. 보낸 트랜잭션만 추출
sent = (
    temp_df[['fromAddress','hour']]
    .rename(columns={'fromAddress':'address'})
)

# 5-3. hour별 보낸 횟수 집계
hourly = (
    sent
    .groupby(['address','hour'])
    .size()
    .rename('hour_count')
    .reset_index()
)

# 5-4. Shannon entropy 계산
def shannon_entropy(counts):
    p = counts / counts.sum()
    return -(p * np.log2(p)).sum()

ent = (
    hourly
    .groupby('address')['hour_count']
    .apply(shannon_entropy)
    .rename('hour_entropy')
    .reset_index()
)
time_stats = ent

# 5-5. 기존 total_stats 와 병합 
total_stats = (
    total_stats.reset_index()
               .merge(time_stats, on='address', how='left')
               .fillna(0)
               .set_index('address')
)

/tmp/ipykernel_115274/3721333743.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp'], unit = 'ms')
/tmp/ipykernel_115274/3721333743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
/tmp/ipykernel_115274/3721333743.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [13]:
#6. 활동 일수 기반 꾸준지표 계산
date_stats = (
    temp_df
    .groupby('fromAddress')['timestamp']
    .agg(
        first_date       = 'min',
        last_date        = 'max',
        active_days_count= lambda x: x.nunique()
    )
    .rename_axis('address')
    .reset_index()
)

total_stats = (
    total_stats.reset_index()
               .merge(date_stats, on='address', how='left')
               .set_index('address')
)
total_stats['active_days_count'] = total_stats['active_days_count'].fillna(0)

In [14]:
#7. 거래 상대의 다양성
# 7-1. 보낸 거래 기준으로 고유 toAddress 수
cp_count_sent = (
    node_df
    .groupby('fromAddress')['toAddress']
    .nunique()
    .rename('counterparty_count_sent')
    .reset_index()
    .rename(columns={'fromAddress':'address'})
)

# 7-2. 받은 거래 기준으로 고유 fromAddress 수
cp_count_recv = (
    node_df
    .groupby('toAddress')['fromAddress']
    .nunique()
    .rename('counterparty_count_recv')
    .reset_index()
    .rename(columns={'toAddress':'address'})
)
# 7-3. 기존 total_stats 와 병합
total_stats = (
    total_stats.reset_index()
               .merge(cp_count_sent, on='address', how='left')
               .merge(cp_count_recv, on='address', how='left')
               .set_index('address')
)
total_stats['counterparty_count_sent'] = total_stats['counterparty_count_sent'].fillna(0)
total_stats['counterparty_count_recv'] = total_stats['counterparty_count_recv'].fillna(0)

In [15]:
total_stats.describe()


,sent_tx_count,sent_tx_amount,recv_tx_count,recv_tx_amount,external_sent_tx_count,external_sent_tx_amount,external_recv_tx_count,external_recv_tx_amount,hour_entropy,active_days_count,counterparty_count_sent,counterparty_count_recv
count,40446.000000,4.044600e+04,40446.00000,4.044600e+04,40446.000000,4.044600e+04,40446.000000,4.044600e+04,40446.000000,40446.000000,40446.000000,40446.000000
mean,1.884710,2.516532e+07,1.88471,2.516532e+07,0.483682,2.808816e+05,0.483682,2.808816e+05,0.066376,1.758493,0.902413,0.902413
std,31.732783,4.972722e+09,37.31266,4.972723e+09,6.131281,5.253131e+07,2.896939,5.255610e+07,0.346828,30.946827,9.467483,23.800939
min,0.000000,0.000000e+00,0.00000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,-0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.00000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.700000e+00,0.00000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,-0.000000,1.000000,1.000000,0.000000
75%,1.000000,8.000000e+00,1.00000,1.101466e+01,1.000000,1.912603e-01,0.000000,0.000000e+00,-0.000000,1.000000,1.000000,1.000000
max,1812.000000,1.000000e+12,4642.00000,1.000000e+12,986.000000,1.055000e+10,309.000000,1.055500e+10,4.439936,1662.000000,1622.000000,4642.000000


In [16]:
#8. 거래금액 특성
# 8-1. 평균 송금액
total_stats['sent_tx_amount_mean'] = total_stats['sent_tx_amount'] / total_stats['sent_tx_count']
# 8-2. 평균 수신액
total_stats['recv_tx_amount_mean'] = total_stats['recv_tx_amount'] / total_stats['recv_tx_count']
# 8-3. 평균 외부 송금액
total_stats['external_sent_tx_amount_mean'] = total_stats['external_sent_tx_amount'] / total_stats['external_sent_tx_count']
# 8-4. 평균 외부 수신액
total_stats['external_recv_tx_amount_mean'] = total_stats['external_recv_tx_amount'] / total_stats['external_recv_tx_count']



In [17]:
total_stats.fillna(0, inplace=True)

/tmp/ipykernel_115274/366128738.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns, Asia/Seoul], please explicitly cast to a compatible dtype first.
  total_stats.fillna(0, inplace=True)


In [19]:
#to csv
total_stats.to_csv("../data/total_stats.csv")

In [20]:
node_df.to_csv("../data/transfer_data_real.csv")

In [21]:
node_df.nunique()

type               3
txhash         76032
timestamp      29761
amount         46728
denom            169
dpDenom           80
fromAddress    28196
toAddress      20197
fromChain         26
fromChainId       26
toChain           27
toChainId         27
is_external        2
hour              22
dtype: int64

In [23]:
node_df.shape

(76229, 14)